# Import Libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

# Split Dataset into Train, Validation, and Testing

In [ ]:
import splitfolders

In [ ]:
splitfolders.ratio("dataset\Mushrooms", output="dataset\dataSplit",
    seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False) # default values

# Load Data

In [ ]:
trainPath = r'dataset\dataSplit\train'
validationPath = r'dataset\dataSplit\val'
testPath = r'dataset\dataSplit\test'

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

#Train
train_generator = train_datagen.flow_from_directory(
                                            trainPath,
                                                    target_size=(300, 300),
                                                    batch_size=64,
                                                    shuffle=True,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                            )

#Validation
validation_generator=val_datagen.flow_from_directory(
                                            validationPath,
                                                    target_size=(300,300),
                                                    batch_size=64,
                                                    shuffle=True,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                )

#Test
test_generator=val_datagen.flow_from_directory(
                                            testPath,
                                                    target_size=(300,300),
                                                    batch_size=64,
                                                    shuffle=True,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                )

# EDA

In [ ]:
# class names
class_names = os.listdir(trainPath)
print('All category : ',class_names)

In [ ]:
#Sampling Dataset
image ,label = next(iter(test_generator))

plt.figure(figsize=(20,10))
for i in range(10) :
  plt.subplot(2,5,i+1)
  plt.imshow(image[i])
  plt.title(class_names[np.argmax(label[i])])
  plt.axis("off")

# CallBacks

In [ ]:
# Callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss',  
                                    factor=0.4, patience=3, 
                                    verbose=1, mode='min', 
                                    min_delta=0.0001, min_lr=0,
                                    restore_best_weights=True)

# Modelling

### Import Arsitektur EfficientNet

In [ ]:
from efficientnet.tfkeras import EfficientNetB0
from efficientnet.tfkeras import EfficientNetB1
from efficientnet.tfkeras import EfficientNetB2
from efficientnet.tfkeras import EfficientNetB3
from efficientnet.tfkeras import EfficientNetB4
from efficientnet.tfkeras import EfficientNetB5
from efficientnet.tfkeras import EfficientNetB6
from efficientnet.tfkeras import EfficientNetB7

### EfficientNet B-0

##### Load Model EfficientNetB0

In [ ]:
pre_trained_model_B0 = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(300, 300, 3),
    pooling='avg',
)

##### Layering EfficeintNetB0

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False


last_layer = pre_trained_model.get_layer('avg_pool')
last_output = last_layer.output


x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)              
x = layers.Dense (9, activation='softmax')(x)           

model_B0 = Model( pre_trained_model_B0.input, x) 

model_B0.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

##### Training Model with EfficientNetB0

In [ ]:
history = model_B0.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 20,
            callbacks=[reduceLROnPlat],
            verbose = 1)  

##### Evaluating Model Using Data Testing

In [ ]:
B0_eval = model_B0.evaluate(test_generator,verbose=0)
B0_acc = round(B0_eval[1],2) * 100
B0_acc

##### Save Model EfficientNetB0

In [ ]:
model_B0.save('saved-models/model_EfficientNetB0.h5')

##### Confusion Matrix B0

In [ ]:
y_test = []
y_pred = []

for i in range(len(test_generator)) :
    x,y = test_generator.next()
    for j in range(len(y)) :
        y_test.append(class_names[tf.argmax(y[j])])
        y_pred.append(class_names[np.argmax(model_B0.predict(x[j][None,...],verbose=0))])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 13),dpi=150)
cm = confusion_matrix(y_test, y_pred)

colors = ["#393E46","#393E46"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

ax.text(0,-0.3,'Confusion Matrix ',fontfamily='serif',fontsize=15,fontweight='bold')
ax.text(0,-0.12,'EfficientB7 Model',fontfamily='serif',fontsize=12,fontweight='light')
sns.heatmap(ax=ax, data=cm,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=1.5, annot=True,cmap=colormap, 
            annot_kws={"fontsize":12},fmt='')

ax.set_xticklabels(class_names)
ax.set_yticklabels(class_names)

ax.set_ylabel("Actual Label",fontsize=10)
ax.set_xlabel("Predicted Label",fontsize=10)

from matplotlib.patches import Rectangle
for i in range(0,8):
    ax.add_patch(Rectangle((i, i), 1, 1, fill=True,color='#00ADB5'))

plt.show()

##### Prediction Image EfficientNetB0

In [ ]:
from keras.models import load_model
import keras.utils as image
import matplotlib.pyplot as plt
import numpy as np

model = load_model('Saved_Models/model_EfficientNetB0.h5')

In [ ]:
B0_eval = model_B0.evaluate(test_generator,verbose=0)
B0_acc = round(B0_eval[1],2) * 100
B0_acc

In [ ]:
img_path = ""
CLASS_NAMES = ['Agglonema', 'Alocasia', 'Gelombang Cinta', 'Janda Bolong', 'Lidah Mertua', 'Lili Paris', 'Pucuk Merah', 'Suplir']

img = image.load_img(img_path, target_size=(275,275))
imgplot = plt.imshow(img)
x = image.img_to_array(img)
x = x/255.0
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

classes = model.predict(images, batch_size=10,verbose=0)
print(f'Hasil Prediksi: {CLASS_NAMES[np.argmax(classes)]}')

### EfficientNet B-1

##### Load Model EfficientNetB1

In [ ]:
pre_trained_model_B1 = EfficientNetB1(
    include_top=False,
    weights="imagenet",
    input_shape=(300, 300, 3),
    pooling='avg',
)

##### Layering EfficientNetB1

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False


last_layer = pre_trained_model.get_layer('avg_pool')
last_output = last_layer.output


x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)              
x = layers.Dense (9, activation='softmax')(x)           

model_B1 = Model( pre_trained_model_B1.input, x) 

model_B1.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

##### Training Model with EfficientNetB1

In [ ]:
history = model_B1.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 20,
            callbacks=[reduceLROnPlat],
            verbose = 1)  

##### Evaluating Model Using Data Testing

In [ ]:
B1_eval = model_B1.evaluate(test_generator,verbose=0)
B1_acc = round(B1_eval[1],2) * 100
B1_acc

##### Save Model EfficientNetB1

In [ ]:
model_B1.save('saved-models/model_EfficientNetB1.h5')

##### Confusion Matrix B1

In [ ]:
y_test = []
y_pred = []

for i in range(len(test_generator)) :
    x,y = test_generator.next()
    for j in range(len(y)) :
        y_test.append(class_names[tf.argmax(y[j])])
        y_pred.append(class_names[np.argmax(model_B1.predict(x[j][None,...],verbose=0))])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 13),dpi=150)
cm = confusion_matrix(y_test, y_pred)

colors = ["#393E46","#393E46"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

ax.text(0,-0.3,'Confusion Matrix ',fontfamily='serif',fontsize=15,fontweight='bold')
ax.text(0,-0.12,'EfficientB7 Model',fontfamily='serif',fontsize=12,fontweight='light')
sns.heatmap(ax=ax, data=cm,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=1.5, annot=True,cmap=colormap, 
            annot_kws={"fontsize":12},fmt='')

ax.set_xticklabels(class_names)
ax.set_yticklabels(class_names)

ax.set_ylabel("Actual Label",fontsize=10)
ax.set_xlabel("Predicted Label",fontsize=10)

from matplotlib.patches import Rectangle
for i in range(0,8):
    ax.add_patch(Rectangle((i, i), 1, 1, fill=True,color='#00ADB5'))

plt.show()

##### Prediction Image B1

In [ ]:
from keras.models import load_model
import keras.utils as image
import matplotlib.pyplot as plt
import numpy as np

model = load_model('Saved_Models/model_EfficientNetB1.h5')

In [ ]:
B1_eval = model_B1.evaluate(test_generator,verbose=0)
B1_acc = round(B1_eval[1],2) * 100
B1_acc

In [ ]:
img_path = ""
CLASS_NAMES = ['Agglonema', 'Alocasia', 'Gelombang Cinta', 'Janda Bolong', 'Lidah Mertua', 'Lili Paris', 'Pucuk Merah', 'Suplir']

img = image.load_img(img_path, target_size=(275,275))
imgplot = plt.imshow(img)
x = image.img_to_array(img)
x = x/255.0
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

classes = model.predict(images, batch_size=10,verbose=0)
print(f'Hasil Prediksi: {CLASS_NAMES[np.argmax(classes)]}')

### EfficientNet B-2

##### Load Model EfficientNetB2

In [ ]:
pre_trained_model_B2 = EfficientNetB2(
    include_top=False,
    weights="imagenet",
    input_shape=(300, 300, 3),
    pooling='avg',
)

##### layering EfficientNetB1

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False


last_layer = pre_trained_model.get_layer('avg_pool')
last_output = last_layer.output


x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)              
x = layers.Dense (9, activation='softmax')(x)           

model_B2 = Model( pre_trained_model_B2.input, x) 

model_B2.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

##### Training Model with EfficientNetB1

In [ ]:
history = model_B2.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 20,
            callbacks=[reduceLROnPlat],
            verbose = 1)  

##### Evaluating Model Using Data Testing

In [ ]:
B2_eval = model_B2.evaluate(test_generator,verbose=0)
B2_acc = round(B2_eval[1],2) * 100
B2_acc

##### Save Model EfficientNetB2

In [ ]:
model_B2.save('saved-models/model_EfficientNetB2.h5')

##### Confusion Matrix B2

In [ ]:
y_test = []
y_pred = []

for i in range(len(test_generator)) :
    x,y = test_generator.next()
    for j in range(len(y)) :
        y_test.append(class_names[tf.argmax(y[j])])
        y_pred.append(class_names[np.argmax(model_B2.predict(x[j][None,...],verbose=0))])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 13),dpi=150)
cm = confusion_matrix(y_test, y_pred)

colors = ["#393E46","#393E46"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

ax.text(0,-0.3,'Confusion Matrix ',fontfamily='serif',fontsize=15,fontweight='bold')
ax.text(0,-0.12,'EfficientB7 Model',fontfamily='serif',fontsize=12,fontweight='light')
sns.heatmap(ax=ax, data=cm,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=1.5, annot=True,cmap=colormap, 
            annot_kws={"fontsize":12},fmt='')

ax.set_xticklabels(class_names)
ax.set_yticklabels(class_names)

ax.set_ylabel("Actual Label",fontsize=10)
ax.set_xlabel("Predicted Label",fontsize=10)

from matplotlib.patches import Rectangle
for i in range(0,8):
    ax.add_patch(Rectangle((i, i), 1, 1, fill=True,color='#00ADB5'))

plt.show()

##### Prediction Image B2

In [ ]:
from keras.models import load_model
import keras.utils as image
import matplotlib.pyplot as plt
import numpy as np

model = load_model('Saved_Models/model_EfficientNetB2.h5')

In [ ]:
B2_eval = model_B2.evaluate(test_generator,verbose=0)
B2_acc = round(B2_eval[1],2) * 100
B2_acc

In [ ]:
img_path = ""
CLASS_NAMES = ['Agglonema', 'Alocasia', 'Gelombang Cinta', 'Janda Bolong', 'Lidah Mertua', 'Lili Paris', 'Pucuk Merah', 'Suplir']

img = image.load_img(img_path, target_size=(275,275))
imgplot = plt.imshow(img)
x = image.img_to_array(img)
x = x/255.0
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

classes = model.predict(images, batch_size=10,verbose=0)
print(f'Hasil Prediksi: {CLASS_NAMES[np.argmax(classes)]}')

### EfficientNet B-3

##### Load Model EfficientNetB3

In [ ]:
pre_trained_model_B3 = EfficientNetB3(
    include_top=False,
    weights="imagenet",
    input_shape=(300, 300, 3),
    pooling='avg',
)

##### Layering EfficientNetB3

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False


last_layer = pre_trained_model.get_layer('avg_pool')
last_output = last_layer.output


x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)              
x = layers.Dense (9, activation='softmax')(x)           

model_B3 = Model( pre_trained_model_B3.input, x) 

model_B3.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

##### Training Model with EfficientNetB3

In [ ]:
history = model_B3.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 20,
            callbacks=[reduceLROnPlat],
            verbose = 1)  

##### Evaluating Model using Data Testing

In [ ]:
B3_eval = model_B3.evaluate(test_generator,verbose=0)
B3_acc = round(B3_eval[1],2) * 100
B3_acc

##### Save Model EfficientNetB3

In [ ]:
model_B3.save('saved-models/model_EfficientNetB3.h5')

##### Confusion Matrix B3

In [ ]:
y_test = []
y_pred = []

for i in range(len(test_generator)) :
    x,y = test_generator.next()
    for j in range(len(y)) :
        y_test.append(class_names[tf.argmax(y[j])])
        y_pred.append(class_names[np.argmax(model_B3.predict(x[j][None,...],verbose=0))])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 13),dpi=150)
cm = confusion_matrix(y_test, y_pred)

colors = ["#393E46","#393E46"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

ax.text(0,-0.3,'Confusion Matrix ',fontfamily='serif',fontsize=15,fontweight='bold')
ax.text(0,-0.12,'EfficientB7 Model',fontfamily='serif',fontsize=12,fontweight='light')
sns.heatmap(ax=ax, data=cm,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=1.5, annot=True,cmap=colormap, 
            annot_kws={"fontsize":12},fmt='')

ax.set_xticklabels(class_names)
ax.set_yticklabels(class_names)

ax.set_ylabel("Actual Label",fontsize=10)
ax.set_xlabel("Predicted Label",fontsize=10)

from matplotlib.patches import Rectangle
for i in range(0,8):
    ax.add_patch(Rectangle((i, i), 1, 1, fill=True,color='#00ADB5'))

plt.show()

##### Prediction Image B3

In [ ]:
from keras.models import load_model
import keras.utils as image
import matplotlib.pyplot as plt
import numpy as np

model = load_model('Saved_Models/model_EfficientNetB3.h5')

In [ ]:
B3_eval = model_B3.evaluate(test_generator,verbose=0)
B3_acc = round(B3_eval[1],2) * 100
B3_acc

In [ ]:
img_path = ""
CLASS_NAMES = ['Agglonema', 'Alocasia', 'Gelombang Cinta', 'Janda Bolong', 'Lidah Mertua', 'Lili Paris', 'Pucuk Merah', 'Suplir']

img = image.load_img(img_path, target_size=(275,275))
imgplot = plt.imshow(img)
x = image.img_to_array(img)
x = x/255.0
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

classes = model.predict(images, batch_size=10,verbose=0)
print(f'Hasil Prediksi: {CLASS_NAMES[np.argmax(classes)]}')

### EfficientNet B-4

##### Load Model EfficientNetB4

In [ ]:
pre_trained_model_B4 = EfficientNetB4(
    include_top=False,
    weights="imagenet",
    input_shape=(300, 300, 3),
    pooling='avg',
)

##### Layering EfficientNetB4

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False


last_layer = pre_trained_model.get_layer('avg_pool')
last_output = last_layer.output


x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)              
x = layers.Dense (9, activation='softmax')(x)           

model_B4 = Model( pre_trained_model_B4.input, x) 

model_B4.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

##### Training Model with EfficientNetB4

In [ ]:
history = model_B4.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 20,
            callbacks=[reduceLROnPlat],
            verbose = 1)  

##### Evaluating Model using Data Testing

In [ ]:
B4_eval = model_B4.evaluate(test_generator,verbose=0)
B4_acc = round(B4_eval[1],2) * 100
B4_acc

##### Save Model EfficientNetB4

In [ ]:
model_B4.save('saved-models/model_EfficientNetB4.h5')

##### Confusion Matrix B4

In [ ]:
y_test = []
y_pred = []

for i in range(len(test_generator)) :
    x,y = test_generator.next()
    for j in range(len(y)) :
        y_test.append(class_names[tf.argmax(y[j])])
        y_pred.append(class_names[np.argmax(model_B4.predict(x[j][None,...],verbose=0))])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 13),dpi=150)
cm = confusion_matrix(y_test, y_pred)

colors = ["#393E46","#393E46"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

ax.text(0,-0.3,'Confusion Matrix ',fontfamily='serif',fontsize=15,fontweight='bold')
ax.text(0,-0.12,'EfficientB7 Model',fontfamily='serif',fontsize=12,fontweight='light')
sns.heatmap(ax=ax, data=cm,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=1.5, annot=True,cmap=colormap, 
            annot_kws={"fontsize":12},fmt='')

ax.set_xticklabels(class_names)
ax.set_yticklabels(class_names)

ax.set_ylabel("Actual Label",fontsize=10)
ax.set_xlabel("Predicted Label",fontsize=10)

from matplotlib.patches import Rectangle
for i in range(0,8):
    ax.add_patch(Rectangle((i, i), 1, 1, fill=True,color='#00ADB5'))

plt.show()

##### Prediction Image B4

In [ ]:
from keras.models import load_model
import keras.utils as image
import matplotlib.pyplot as plt
import numpy as np

model = load_model('Saved_Models/model_EfficientNetB4.h5')

In [ ]:
B4_eval = model_B4.evaluate(test_generator,verbose=0)
B4_acc = round(B4_eval[1],2) * 100
B4_acc

In [ ]:
img_path = ""
CLASS_NAMES = ['Agglonema', 'Alocasia', 'Gelombang Cinta', 'Janda Bolong', 'Lidah Mertua', 'Lili Paris', 'Pucuk Merah', 'Suplir']

img = image.load_img(img_path, target_size=(275,275))
imgplot = plt.imshow(img)
x = image.img_to_array(img)
x = x/255.0
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

classes = model.predict(images, batch_size=10,verbose=0)
print(f'Hasil Prediksi: {CLASS_NAMES[np.argmax(classes)]}')

### EfficientNet B-5

##### Load Model EfficientNetB5

In [ ]:
pre_trained_model_B5 = EfficientNetB5(
    include_top=False,
    weights="imagenet",
    input_shape=(300, 300, 3),
    pooling='avg',
)

##### layering EfficientNetB5

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False


last_layer = pre_trained_model.get_layer('avg_pool')
last_output = last_layer.output


x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)              
x = layers.Dense (9, activation='softmax')(x)           

model_B5 = Model( pre_trained_model_B5.input, x) 

model_B5.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

##### Training Model with EfficientNetB5

In [ ]:
history = model_B5.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 20,
            callbacks=[reduceLROnPlat],
            verbose = 1)  

##### Evaluating Model using Data Testing

In [ ]:
B5_eval = model_B5.evaluate(test_generator,verbose=0)
B5_acc = round(B5_eval[1],2) * 100
B5_acc

##### Save Model EfficientNetB5

In [ ]:
model_B5.save('saved-models/model_EfficientNetB5.h5')

##### Confusion Matrix B5

In [ ]:
y_test = []
y_pred = []

for i in range(len(test_generator)) :
    x,y = test_generator.next()
    for j in range(len(y)) :
        y_test.append(class_names[tf.argmax(y[j])])
        y_pred.append(class_names[np.argmax(model_B5.predict(x[j][None,...],verbose=0))])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 13),dpi=150)
cm = confusion_matrix(y_test, y_pred)

colors = ["#393E46","#393E46"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

ax.text(0,-0.3,'Confusion Matrix ',fontfamily='serif',fontsize=15,fontweight='bold')
ax.text(0,-0.12,'EfficientB7 Model',fontfamily='serif',fontsize=12,fontweight='light')
sns.heatmap(ax=ax, data=cm,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=1.5, annot=True,cmap=colormap, 
            annot_kws={"fontsize":12},fmt='')

ax.set_xticklabels(class_names)
ax.set_yticklabels(class_names)

ax.set_ylabel("Actual Label",fontsize=10)
ax.set_xlabel("Predicted Label",fontsize=10)

from matplotlib.patches import Rectangle
for i in range(0,8):
    ax.add_patch(Rectangle((i, i), 1, 1, fill=True,color='#00ADB5'))

plt.show()

##### Prediction Image B5

In [ ]:
from keras.models import load_model
import keras.utils as image
import matplotlib.pyplot as plt
import numpy as np

model = load_model('Saved_Models/model_EfficientNetB5.h5')

In [ ]:
B5_eval = model_B5.evaluate(test_generator,verbose=0)
B5_acc = round(B5_eval[1],2) * 100
B5_acc

In [ ]:
img_path = ""
CLASS_NAMES = ['Agglonema', 'Alocasia', 'Gelombang Cinta', 'Janda Bolong', 'Lidah Mertua', 'Lili Paris', 'Pucuk Merah', 'Suplir']

img = image.load_img(img_path, target_size=(275,275))
imgplot = plt.imshow(img)
x = image.img_to_array(img)
x = x/255.0
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

classes = model.predict(images, batch_size=10,verbose=0)
print(f'Hasil Prediksi: {CLASS_NAMES[np.argmax(classes)]}')

### EfficientNet B-6

##### Load Model EfficientNetB6

In [ ]:
pre_trained_model_B6 = EfficientNetB6(
    include_top=False,
    weights="imagenet",
    input_shape=(300, 300, 3),
    pooling='avg',
)

##### Layering EfficientNetB6

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False


last_layer = pre_trained_model.get_layer('avg_pool')
last_output = last_layer.output


x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)              
x = layers.Dense (9, activation='softmax')(x)           

model_B6 = Model( pre_trained_model_B6.input, x) 

model_B6.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

##### Training Model with EfficientNetB6

In [ ]:
history = model_B6.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 20,
            callbacks=[reduceLROnPlat],
            verbose = 1)  

##### Evaluating Model using Data Testing

In [ ]:
B6_eval = model_B6.evaluate(test_generator,verbose=0)
B6_acc = round(B6_eval[1],2) * 100
B6_acc

##### Save Model using Data Testing

In [ ]:
model_B6.save('saved-models/model_EfficientNetB6.h5')

##### Confusion Matrix B6

In [ ]:
y_test = []
y_pred = []

for i in range(len(test_generator)) :
    x,y = test_generator.next()
    for j in range(len(y)) :
        y_test.append(class_names[tf.argmax(y[j])])
        y_pred.append(class_names[np.argmax(model_B6.predict(x[j][None,...],verbose=0))])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 13),dpi=150)
cm = confusion_matrix(y_test, y_pred)

colors = ["#393E46","#393E46"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

ax.text(0,-0.3,'Confusion Matrix ',fontfamily='serif',fontsize=15,fontweight='bold')
ax.text(0,-0.12,'EfficientB7 Model',fontfamily='serif',fontsize=12,fontweight='light')
sns.heatmap(ax=ax, data=cm,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=1.5, annot=True,cmap=colormap, 
            annot_kws={"fontsize":12},fmt='')

ax.set_xticklabels(class_names)
ax.set_yticklabels(class_names)

ax.set_ylabel("Actual Label",fontsize=10)
ax.set_xlabel("Predicted Label",fontsize=10)

from matplotlib.patches import Rectangle
for i in range(0,8):
    ax.add_patch(Rectangle((i, i), 1, 1, fill=True,color='#00ADB5'))

plt.show()

##### Prediction Image B6

In [ ]:
from keras.models import load_model
import keras.utils as image
import matplotlib.pyplot as plt
import numpy as np

model = load_model('Saved_Models/model_EfficientNetB6.h5')

In [ ]:
B6_eval = model_B6.evaluate(test_generator,verbose=0)
B6_acc = round(B6_eval[1],2) * 100
B6_acc

In [ ]:
img_path = ""
CLASS_NAMES = ['Agglonema', 'Alocasia', 'Gelombang Cinta', 'Janda Bolong', 'Lidah Mertua', 'Lili Paris', 'Pucuk Merah', 'Suplir']

img = image.load_img(img_path, target_size=(275,275))
imgplot = plt.imshow(img)
x = image.img_to_array(img)
x = x/255.0
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

classes = model.predict(images, batch_size=10,verbose=0)
print(f'Hasil Prediksi: {CLASS_NAMES[np.argmax(classes)]}')

### EfficientNet B-7

##### Load Model EfficientNetB7

In [ ]:
pre_trained_model_B7 = EfficientNetB7(
    include_top=False,
    weights="imagenet",
    input_shape=(300, 300, 3),
    pooling='avg',
)

##### Layering EfficientNetB7

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False


last_layer = pre_trained_model.get_layer('avg_pool')
last_output = last_layer.output


x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)              
x = layers.Dense (9, activation='softmax')(x)           

model_B7 = Model( pre_trained_model_B7.input, x) 

model_B7.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

##### Training Model with EfficientNetB7

In [ ]:
history = model_B7.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 20,
            callbacks=[reduceLROnPlat],
            verbose = 1)  

##### Evaluating Model using Data Testing

In [ ]:
B7_eval = model_B7.evaluate(test_generator,verbose=0)
B7_acc = round(B7_eval[1],2) * 100
B7_acc

##### Save Model using Data Testing

In [ ]:
model_B7.save('saved-models/model_EfficientNetB7.h5')

##### Confusion Matrix B7

In [ ]:
y_test = []
y_pred = []

for i in range(len(test_generator)) :
    x,y = test_generator.next()
    for j in range(len(y)) :
        y_test.append(class_names[tf.argmax(y[j])])
        y_pred.append(class_names[np.argmax(model_B7.predict(x[j][None,...],verbose=0))])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 13),dpi=150)
cm = confusion_matrix(y_test, y_pred)

colors = ["#393E46","#393E46"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

ax.text(0,-0.3,'Confusion Matrix ',fontfamily='serif',fontsize=15,fontweight='bold')
ax.text(0,-0.12,'EfficientB7 Model',fontfamily='serif',fontsize=12,fontweight='light')
sns.heatmap(ax=ax, data=cm,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=1.5, annot=True,cmap=colormap, 
            annot_kws={"fontsize":12},fmt='')

ax.set_xticklabels(class_names)
ax.set_yticklabels(class_names)

ax.set_ylabel("Actual Label",fontsize=10)
ax.set_xlabel("Predicted Label",fontsize=10)

from matplotlib.patches import Rectangle
for i in range(0,8):
    ax.add_patch(Rectangle((i, i), 1, 1, fill=True,color='#00ADB5'))

plt.show()

##### Prediction Image B7

In [ ]:
from keras.models import load_model
import keras.utils as image
import matplotlib.pyplot as plt
import numpy as np

model = load_model('Saved_Models/model_EfficientNetB7.h5')

In [ ]:
B7_eval = model_B7.evaluate(test_generator,verbose=0)
B7_acc = round(B7_eval[1],2) * 100
B7_acc

In [ ]:
img_path = ""
CLASS_NAMES = ['Agglonema', 'Alocasia', 'Gelombang Cinta', 'Janda Bolong', 'Lidah Mertua', 'Lili Paris', 'Pucuk Merah', 'Suplir']

img = image.load_img(img_path, target_size=(275,275))
imgplot = plt.imshow(img)
x = image.img_to_array(img)
x = x/255.0
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

classes = model.predict(images, batch_size=10,verbose=0)
print(f'Hasil Prediksi: {CLASS_NAMES[np.argmax(classes)]}')